#### Feature Engineering & Encoding.

Esto lo ponemos en un notebook aparte porque a partir de acá se crearán Pipelines para los datos, además, se crearán los artifacts necesarios a usar en el despliegue.

**Feature Engineering tasks:** 

- Geohashes.
- Distritos con más ingresos.
- Precio por m2 en Geohashes (KNN para los por inputar)

**Hacer encodings**

**Ver Correlaciones**

**Ver distribuciones por categorías**

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import geohash2 as gh #pip install geohash2
import gc

lima_data = pd.read_csv('../data/2020_Notebook01_clean_data.csv')
lima_data.describe()

#Las coordenadas son negativas, se scrapearon sin el -1
lima_data['latitud'] = lima_data['latitud']*-1
lima_data['longitud'] = lima_data['longitud']*-1

#### GeoHashes

In [24]:
for num in range(5, 10):
    lima_data['geohash_grado{}'.format(num)] = lima_data.apply(lambda x: gh.encode(x['latitud'], x['longitud'], precision = num),axis=1)

In [25]:
top_geohashes = lima_data['geohash_grado7'].value_counts().iloc[0:7].keys().tolist()

#En qué Distrito están los barrios con mayor oferta?
lima_data[lima_data['geohash_grado7'].isin(top_geohashes)]['Barrio'].value_counts()

La Molina            262
Santiago de Surco     32
San Isidro            28
Miraflores             8
NoEspecifica           6
Lima Cercado           1
Barranco               1
Ate Vitarte            1
La Victoria            1
San Borja              1
Name: Barrio, dtype: int64

#### OJO! Varios anuncios no ponen las coordenadas exactas, sino unas aledañas. Por eso se ve como si este barrio tuviera varias casas en oferta. Son las mismas 78 coordenadas!! Pero diferentes casas. Todas en la avenida La Molina, muy cerca a Molicentro. Igual pasa en Surco y en San isidro pero con algo de menos frecuencia. 

In [26]:
lima_data[lima_data['geohash_grado7'] == top_geohashes[0]][['latitud','longitud']].value_counts()

latitud     longitud  
-12.082020  -76.928234    78
-12.081859  -76.928393     2
            -76.928393     2
-12.082020  -76.928234     2
-12.081075  -76.928427     1
-12.081714  -76.927820     1
-12.081852  -76.928062     1
-12.082040  -76.928287     1
dtype: int64

In [27]:
print(lima_data[lima_data['geohash_grado7'] == top_geohashes[1]][['latitud','longitud']].value_counts())
print(lima_data[lima_data['geohash_grado7'] == top_geohashes[2]][['latitud','longitud']].value_counts())

latitud     longitud  
-12.096333  -77.028072    88
dtype: int64
latitud     longitud  
-12.074293  -76.913630    29
-12.074288  -76.913623     2
-12.074000  -76.913247     1
-12.074250  -76.913516     1
-12.074288  -76.913623     1
-12.074651  -76.913879     1
-12.074677  -76.913869     1
-12.075320  -76.913009     1
dtype: int64


In [41]:
#Solo los objects a encodear. Para los Geohash la misma librería tiene su feature de encode. 
lima_data.select_dtypes(include='object').filter(regex='^(?!geohash*)')

,Barrio,Ciudad,MONEDA,Precio_cat,Area_constr_cat
0,La Molina,Lima,S/,medio,"(787.525, 1574.05]"
1,La Molina,Lima,S/,muy caro,"(787.525, 1574.05]"
2,La Molina,Lima,S/,caro,"(787.525, 1574.05]"
3,La Molina,Lima,S/,caro,"(787.525, 1574.05]"
4,La Molina,Lima,S/,caro,"(787.525, 1574.05]"
...,...,...,...,...,...
5735,Jesús María,Lima,USD,muy caro,"(787.525, 1574.05]"
5736,Chaclacayo,Lima,S/,bajo,"(1574.05, 2360.575]"
5737,NoEspecifica,Lima,S/,medio,"(3147.1, 3933.625]"
5738,San Juan de Lurigancho,Lima,USD,bajo,"(7866.25, 8652.775]"
